* https://medium.com/analytics-vidhya/generating-a-dessert-ingredients-list-12edd1740753#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImFjYjZiZTUxZWZlYTZhNDE5ZWM5MzI1ZmVhYTFlYzQ2NjBmNWIzN2MiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2NDU3NTE3MjYsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjExMjg5ODkxMzg4OTk3MTcxMDQ0OSIsImVtYWlsIjoia2lya2VuZGFsbC5ldmVAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImF6cCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsIm5hbWUiOiJFdmVseW4gS2lya2VuZGFsbCIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQVRYQUp5Y3lzVGVLUkdBOS1EcW40dnNsZWFZb0VfNnVkV1Q5WW9qcFBTMD1zOTYtYyIsImdpdmVuX25hbWUiOiJFdmVseW4iLCJmYW1pbHlfbmFtZSI6IktpcmtlbmRhbGwiLCJpYXQiOjE2NDU3NTIwMjYsImV4cCI6MTY0NTc1NTYyNiwianRpIjoiNzM3ZDEzNWU2NTk0YzZmNTZlMzNhNjJjYWUyNzZkMzk0ZmNkOWNmYiJ9.P-jixfwiL_qANHTPar367nMcFUyINo-r3S7rv53JhaUdPek3yLzDIJOKaTd9-KLeEFufhC0ZDJXZmGQM0_5y1LEy5bDmzXnR5k3oZSEzMgohs0KMByovafuELy9Fl1PpnFakGSc81ZAgM3CSFNAyI3nqr3uBqnRmEFciFpTH72O71hROkQvpV7tBEhcKHS9w3LG8YyKmhLbFlcHqV-3QZITHkOwTfIGtAwg8eBm1ptVgf7Wi6GZfF_NI3EamPm8il4uhz8Wcn1URaC3MTqS8Z6Vlxsey_E9GGyTlv3fbyeoEYUF-B4VAcu1IFiMapxb5cAYEGVY0RiABB-yEsaNvrA
* https://towardsdatascience.com/using-machine-learning-to-generate-recipes-that-actually-works-b2331c85ab72

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd /home/ec2-user/SageMaker/just-a-recipe-generator

/home/ec2-user/SageMaker/just-a-recipe-generator


In [3]:
# !pip install tensorflow==2.8.0

In [57]:
import os
import time
import copy
import tensorflow as tf

In [33]:
from src.data.pickling import load_pickle
from src.features.clean_shoestring_data import clean_shoestring_recipes
from src.features.prepare_model_data import (ingredients_to_text, tokenize_text,
                                             create_sequences, split_input_target,
                                             make_training_data, text_from_ids)

In [73]:
from src.models.rnn import train_rnn_model, apply_one_step_model

In [6]:
shoestring_recipes0 = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [7]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [8]:
recipe_dict = clean_shoestring_recipes(shoestring_recipes)

In [30]:
text = ingredients_to_text(recipe_dict)
vocab, ids_from_chars, chars_from_ids = tokenize_text(text)
sequences = create_sequences(text, ids_from_chars)
dataset0 = sequences.map(split_input_target)
dataset = make_training_data(dataset0)

In [76]:
history, model = train_rnn_model(dataset, ids_from_chars, vocab, epochs=3)
apply_one_step_model(model, chars_from_ids, ids_from_chars)

Epoch 1/20
8/8 [==============================] - 27s 3s/step - loss: 4.2418
Epoch 2/20
8/8 [==============================] - 24s 3s/step - loss: 4.1033
Epoch 3/20
8/8 [==============================] - 26s 3s/step - loss: 3.5450
Epoch 4/20
8/8 [==============================] - 24s 3s/step - loss: 2.9741
Epoch 5/20
8/8 [==============================] - 25s 3s/step - loss: 2.7141
Epoch 6/20
8/8 [==============================] - 25s 3s/step - loss: 2.4332
Epoch 7/20
8/8 [==============================] - 25s 3s/step - loss: 2.1991
Epoch 8/20
8/8 [==============================] - 26s 3s/step - loss: 2.0186
Epoch 9/20
8/8 [==============================] - 24s 3s/step - loss: 1.8815
Epoch 10/20
8/8 [==============================] - 24s 3s/step - loss: 1.7781
Epoch 11/20
8/8 [==============================] - 24s 3s/step - loss: 1.6700
Epoch 12/20
8/8 [==============================] - 24s 3s/step - loss: 1.5844
Epoch 13/20
8/8 [==============================] - 25s 3s/step - loss: 1.

In [ ]:
# Generate recipe title first? And from that, generate ingredients?
# https://www.tensorflow.org/guide/keras/rnn#rnns_with_listdict_inputs_or_nested_inputs

## Create training batches

Before feeding this data into the model, you need to shuffle the data and pack it into batches.

## Build The Model
This section defines the model as a keras.Model subclass.

This model has three layers:

- tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

### References:

https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [14]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [60]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [36]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

print(sampled_indices)

print("Input:\n", text_from_ids(input_example_batch[0], chars_from_ids))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices, chars_from_ids))

[60 61 63 41 78 41 48 23 13 55  0 67 61 26  0 45 38 56 42 22 35 69 28 32
 32 47 61 32 15 76 24 34  9 43 44 67 52 73 17 70 43 30 18 45 20 51 69 76
 68 14 10 33 51 46 51 53  1 48 43 42  4  8 67  6 77 63 76 35 21 12 10 56
 53 10 77 16  9 19 13 35 79 63 38 42 60  3 19 27 35 18 13 31 57  3 51 75
 70 15 15 55]
Input:
 b'.75 cups all purpose gluten free flour blend\n 1.5 teaspoons xanthan gum\n 0.5 cup tapioca starch/flou'

Next Char Predictions:
 b'oprW\xe2\x85\x93WcC4j[UNK]vpF[UNK]]TkYBPxHLLbpL6\xc3\xa8DN0Z[vg\xc2\xbc8yZJ9];fx\xc3\xa8w51Mfafh\ncZY*/v-\xe2\x80\x99r\xc3\xa8PA31kh1\xe2\x80\x9970:4P\xe2\x85\x9brTYo&:GP94Kl&f\xc2\xbey66j'


In [61]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [38]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 80)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.383031, shape=(), dtype=float32)


In [39]:
# check that the exponential of the mean loss is approximately equal to the vocabulary size.
# A much higher loss means the model is sure of its wrong answers, and is badly initialized
tf.exp(example_batch_mean_loss).numpy()

80.08038

In [62]:
model.compile(optimizer='adam', loss=loss)

In [47]:
# save checkpoints during training'
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "checkpoint_{epoch}")

os.makedirs(checkpoint_dir, exist_ok=True)

In [49]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [63]:
EPOCHS = 50

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
8/8 [==============================] - 27s 3s/step - loss: 4.2331
Epoch 2/50
8/8 [==============================] - 25s 3s/step - loss: 4.0945
Epoch 3/50
8/8 [==============================] - 27s 3s/step - loss: 3.4962
Epoch 4/50
8/8 [==============================] - 24s 3s/step - loss: 2.9806
Epoch 5/50
8/8 [==============================] - 25s 3s/step - loss: 2.7190
Epoch 6/50
8/8 [==============================] - 25s 3s/step - loss: 2.4382
Epoch 7/50
8/8 [==============================] - 25s 3s/step - loss: 2.2114
Epoch 8/50
8/8 [==============================] - 25s 3s/step - loss: 2.0263
Epoch 9/50
8/8 [==============================] - 25s 3s/step - loss: 1.8826
Epoch 10/50
8/8 [==============================] - 25s 3s/step - loss: 1.7721
Epoch 11/50
8/8 [==============================] - 25s 3s/step - loss: 1.6682
Epoch 12/50
8/8 [==============================] - 26s 3s/step - loss: 1.5569
Epoch 13/50
8/8 [==============================] - 25s 3s/step - loss: 1.

In [64]:
class OneStep(tf.keras.Model):
    """Make a single step prediction"""
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [65]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [66]:
start = time.time()
states = None
next_char = tf.constant(['Ingredients:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Ingredients:
 3.33 cups extra tian gruten purce canafle dark Bread milk

 Ingredients:
 0.75 cup granulated sugar
 0.75 cup granulated sugar
 1.0 egg
 1.0 egg whites
 1.0 cup all purpose gluten-free plour blend
 12.0 tablespoons unsalted butter
 Lukewarm water by the 0.5 teaspoon ground ciniamps butter
 2.0 tablespoons & 310 ggum
 0.35 cup um werteri
 1.0 9.0 ounces cedtsated sugar
 2.0 eggs
 1.5 cups milk
 4.0 ounces unsweetened chocolate
 8.0 ounces seats chld exdrued riaces
 2.0 teaspoons pure vanilla extract
 Seeaspoon xanthan gum
 2.0 tablespoons unsalted butter

 Ingredients:
 1.75 cups all-purpose gluten free flour blend
 1.5 teaspoons baking powder
 0.5 teaspoon kosher salt
 2.0 teaspoons pure maple syrup
 0.25 cup nuter
 4.0 tablespoons unsulphured molasses
 0.5 cup packed light brown sugar
 1.0 egg
 1.0 tablespoon granulated sugar
 1.25 cups bloth rocolate chips
 3.0 ounces enspack ole-sweetened chocolate

 Ingredients:
 1.51cup glanu cupfree gruten free pasalt grated corndra

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [77]:
apply_one_step_model(model, chars_from_ids, ids_from_chars)

Ingredients: ons
 1.0 tablespoons unsalted butthre froon bakinga
 0.5 teaspoon froundstar bated
 0.5 teaspoons pash baamed sheegly
  grawterpinited sugar
 5.75 teaspoons varing poweem glack
 0.5 teaspoon kosher salt
 0.5 p ] uras graked bietspor
 0.0 tablespoon superdine frour chetpy-suct
 1.5 tablespoons waniel sure froun baking soda
 6.0 egg
 1.2 teaspoon kosher oant yil

 3.0 gables outter shion
 0.5 teaspoon baking soda
 0.12 t blaspoons yilind piundres batt
 0.25 teaspoon kosh poanendi[ taried soanstare vamily


 Ingredients:
 61.0 sugar
 1.55 gem
 1.0 gur
 0.75 cup orens ounce
 1.0 tablespoons unsweeterter sugar
 0.0 tu7 loun yan fate coonstoonster batieg powder-Dis
 1.0 teaspoon xanthant ctareg
 0.25 teaspoons unsalted butter
 1.0 peaspion parcer salt
 1.0 tablespoons doterarmosa 0.5 cup our
 8.0 tablespoons gi]d toamsves
 0.75 cup w8.25 cup garp sales

 Ingredients:
 2.0 tablespoons granu
 0.5 cup packid poras all purspoond cornitict
 4.0 8alled outterm [ terspoon xabthanets
 3

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control. It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement curriculum learning to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

model.fit(dataset, epochs=1)